### 使用高层API

In [2]:
"""保存动态图模型

保存动态图模型的两种方式：
- 开启训练时调用的paddle.Model.fit函数可自动保存模型，通过它的参数 save_freq可以设置保存动态图模型的频率，
    即多少个 epoch 保存一次模型，默认值是 1
- 调用 paddle.Model.saveAPI，只需要传入保存的模型文件的前缀，即可保存训练后的模型参数和优化器参数，
    保存后的文件后缀名固定为 .pdparams 和.pdopt。
"""

import paddle
import paddle.nn as nn
import paddle.vision.transforms as T
from paddle.vision.models import LeNet

model = paddle.Model(LeNet())
optim = paddle.optimizer.SGD(learning_rate=1e-3,
    parameters=model.parameters())
model.prepare(optim, paddle.nn.CrossEntropyLoss())

transform = T.Compose([
    T.Transpose(),
    T.Normalize([127.5], [127.5])
])
data = paddle.vision.datasets.MNIST(mode='train', transform=transform)

#方式一：设置训练过程中保存模型
# model.fit(data, epochs=1, batch_size=32, save_freq=1)

#方式二：设置训练后保存模型
model.save('../models/test')

In [3]:
"""加载动态图模型

高层 API 加载动态图模型所需要调用的 API 是 paddle.Model.load，从指定的文件中载入模型参数和优化器参数（可选）以继续训练。
paddle.Model.load需要传入的核心的参数是待加载的模型参数或者优化器参数文件（可选）的前缀（需要保证后缀符合 .pdparams 和.pdopt
"""

import paddle
import paddle.nn as nn
import paddle.vision.transforms as T
from paddle.vision.models import LeNet

model = paddle.Model(LeNet())
optim = paddle.optimizer.SGD(learning_rate=1e-3,
    parameters=model.parameters())
model.prepare(optim, paddle.nn.CrossEntropyLoss())

transform = T.Compose([
    T.Transpose(),
    T.Normalize([127.5], [127.5])
])
data = paddle.vision.datasets.MNIST(mode='train', transform=transform)

# 加载模型参数和优化器参数
model.load('../models/test')
model.fit(data, epochs=1, batch_size=32, save_freq=1)

model.save('../models/test_freq') 

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/1
step   10/1875 - loss: 2.7796 - 24ms/step
step   20/1875 - loss: 2.3775 - 21ms/step
step   30/1875 - loss: 2.3387 - 21ms/step
step   40/1875 - loss: 2.2269 - 22ms/step
step   50/1875 - loss: 2.1822 - 24ms/step
step   60/1875 - loss: 2.3911 - 24ms/step
step   70/1875 - loss: 2.1516 - 25ms/step
step   80/1875 - loss: 2.1721 - 25ms/step
step   90/1875 - loss: 2.0140 - 26ms/step
step  100/1875 - loss: 2.1766 - 25ms/step
step  110/1875 - loss: 2.1561 - 26ms/step
step  120/1875 - loss: 2.0069 - 26ms/step
step  130/1875 - loss: 1.9215 - 25ms/step
step  140/1875 - loss: 1.6938 - 26ms/step
step  150/1875 - loss: 1.8043 - 26ms/step
step  160/1875 - loss: 1.6594 - 26ms/step
step  170/1875 - loss: 1.6236 - 26ms/step
step  180/1875 - loss: 1.7618 - 26ms/step
step  190/1875 - loss: 1.7516 - 26ms/step
step  200/1875 - loss: 1.5499 - 26ms/step
step  210/1875 - loss: 1.7939 - 26ms/ste

## 用于推理部署场景

由于动态图模型采用 Python 实时执行的方式，开销较大，在性能方面与 C++ 有一定差距；静态图模型将前端 Python 编写的神经网络预定义为 Program 描述，转到 C++ 端重新解析执行，脱离了 Python 依赖，往往执行性能更佳，并且预先拥有完整网络结构也更利于全局优化，在推理部署场景有天然的优势。

在飞桨框架中，动态图模型训练完成后，为了在部署场景中获得更好的推理性能，提供了自动将动态图模型保存为静态图模型的功能，主要使用的保存和加载 API 是 paddle.jit.save 和 paddle.jit.load。

### 使用高层API

高层 API paddle.Model.save可支持保存推理使用的模型，此时高层 API 在动态图下实际上是对paddle.jit.save的封装，在静态图下是对 paddle.static.save_inference_model的封装，会自动将训练好的动态图模型保存为静态图模型。

In [5]:
"""保存推理使用模型

- paddle.Model.save的第一个参数需要设置为待保存的模型和参数等文件的前缀名
- 第二个参数 training 表示是否保存动态图模型以继续训练，默认是 True，这里需要设为 False，即保存推理部署所需的参数与文件

保存静态图模型会生成三个文件：
*.pdmodel         保存模型的网络结构
*.pdiparams       保存模型中所有的权重数据
*.pdiparams.info  保存和参数状态有关的额外信息
"""

model.save('../models/inference_model', False)  # save for inference